In [51]:
import numpy as np
import pickle
import json
from collections import defaultdict
import re
from pprint import pprint
from dateutil.parser import parse

In [2]:
with open("reviewratings_iclr17_peeread.pkl", "rb") as f:
    rr = pickle.load(f)

In [3]:
type(rr)

dict

In [4]:
rr.keys()

dict_keys(['438', '705', '383', '627', '405', '590', '380', '421', '374', '442', '308', '379', '612', '436', '697', '525', '552', '529', '396', '707', '770', '751', '435', '389', '651', '348', '358', '344', '785', '376', '523', '448', '537', '629', '449', '322', '553', '719', '573', '526', '636', '640', '441', '781', '499', '534', '592', '532', '763', '644', '551', '413', '507', '700', '641', '369', '581', '496', '319', '362', '654', '608', '384', '680', '474', '572', '691', '504', '373', '606', '730', '632', '657', '433', '634', '663', '345', '306', '452', '582', '515', '637', '706', '517', '304', '734', '530', '544', '338', '783', '681', '741', '668', '563', '328', '446', '579', '316', '406', '594', '555', '667', '658', '411', '464', '653', '493', '494', '709', '720', '749', '774', '485', '574', '483', '567', '462', '633', '400', '735', '451', '789', '578', '609', '513', '540', '444', '665', '447', '487', '360', '390', '508', '356', '728', '733', '365', '689', '725', '343', '688', '6

In [5]:
rr['312'].keys(), rr['612']['accepted']

(dict_keys(['title', 'authors', 'abstract', 'conference', 'accepted', 'id', 'reviews', 'histories']),
 False)

In [6]:
rr['312']['reviews'][1]

{'DATE': '19 May 2017 (modified: 02 Jun 2017)',
 'IS_ANNOTATED': True,
 'IS_META_REVIEW': False,
 'ORIGINALITY': 4,
 'OTHER_KEYS': 'ICLR 2017 conference AnonReviewer2',
 'RECOMMENDATION_UNOFFICIAL': 5,
 'SOUNDNESS_CORRECTNESS': 5,
 'TITLE': 'Hyperparameters for SotA PTB word level LM',
 'comments': 'The paper reports that "[a]fter the controller RNN is done training, we take the best RNN cell according to the lowest validation perplexity and then run a grid search over learning rate, weight initialization, dropout rates and decay epoch. The best cell found was then run with three different configurations and sizes to increase its capacity."\n\nIs it possible for you to include (or provide here) the hyperparameters and type of dropout (i.e. recurrent dropout, embedding dropout, ...) used? Without them, replication would at best require a great deal of trial and error. As with "Recurrent Neural Network Regularization" (Zaremba et al., 2014), releasing a base set of hyper parameters great

In [7]:
uniq = []

for k in rr:
    if "reviews" in rr[k]:
        for rev in rr[k]["reviews"]:
            if "OTHER_KEYS" in rev:
                uniq = uniq + list(rev.keys())
                uniq = list(set(uniq))
print(uniq)

['SOUNDNESS_CORRECTNESS', 'CLARITY', 'RECOMMENDATION', 'ORIGINALITY', 'IS_META_REVIEW', 'TITLE', 'IMPACT', 'RECOMMENDATION_UNOFFICIAL', 'REVIEWER_CONFIDENCE', 'DATE', 'comments', 'OTHER_KEYS', 'IS_ANNOTATED', 'SUBSTANCE', 'MEANINGFUL_COMPARISON', 'APPROPRIATENESS']


In [29]:
acl_aspects = ["MEANINGFUL_COMPARISON", "IMPACT", "ORIGINALITY", "RECOMMENDATION", "SUBSTANCE", "SOUNDNESS_CORRECTNESS", "APPROPRIATENESS", "CLARITY"]

aspect_dict = defaultdict(dict)
inspect_list = []

for k in rr:
    new_key = rr[k]["title"].strip().lower()
    
    new_key = re.sub('[^0-9a-zA-Z]+', '', new_key)
    
    aspect_dict[new_key]["title"] = rr[k]["title"]
    aspect_dict[new_key]["dec"] = rr[k]["accepted"]
    aspect_dict[new_key]["pr_id"] = k
    if "reviews" in rr[k]:
        for rev in rr[k]["reviews"]:
            if "OTHER_KEYS" in rev:
                if rev["OTHER_KEYS"].find("AnonReviewer") > -1:
                    for asp in acl_aspects:
                        if asp in rev:
                            if asp in aspect_dict[new_key]:
                                aspect_dict[new_key][asp].append(rev[asp])
                            else:
                                aspect_dict[new_key][asp] = [rev[asp]]
            else:
                for asp in acl_aspects:
                    if asp in rev:
                        inspect_list.append((k, rev))

### Fixing multiple redundant comments

In [97]:
acl_aspects = ["MEANINGFUL_COMPARISON", "IMPACT", "ORIGINALITY", "RECOMMENDATION", "SUBSTANCE", "SOUNDNESS_CORRECTNESS", "APPROPRIATENESS", "CLARITY"]

aspect_dict = defaultdict(dict)
inspect_list = {}

for k in rr:
    new_key = rr[k]["title"].strip().lower()
    
    new_key = re.sub('[^0-9a-zA-Z]+', '', new_key)
    
    aspect_dict[new_key]["title"] = rr[k]["title"]
    aspect_dict[new_key]["dec"] = rr[k]["accepted"]
    aspect_dict[new_key]["pr_id"] = k
    
    inspect_list[new_key] = {}
    
    if "reviews" in rr[k]:
        for rev in rr[k]["reviews"]:
            if "OTHER_KEYS" in rev:
                if rev["OTHER_KEYS"].find("AnonReviewer") > -1:
                    anon_reviewer = rev["OTHER_KEYS"].split(" ")[-1]
                    if anon_reviewer in inspect_list[new_key]:
                        inspect_list[new_key][anon_reviewer]
                    else:
                        inspect_list[new_key][anon_reviewer] = {}
                        if "DATE" in rev:
                            if rev["DATE"].find("modified") > -1:
                                try:
                                    rev["DATE"] = re.findall('([0-9]+ [A-Za-z]+ [0-9][0-9][0-9][0-9]\))$', d)[0][:-1]
                                except Exception:
                                    print(rev["DATE"])
                            else:
                                try:
                                    rev["DATE"] = rev["DATE"].replace("(", "")
                                    rev["DATE"] = rev["DATE"].replace(")", "")
                                    date_of_review = parse(rev["DATE"])
                                except Exception:
                                    print(rev["DATE"], k)
                        else:
                            date_of_review = parse("1 Jan 2000")
                    
                        for asp in acl_aspects:
                            if asp in rev:
                                if asp in inspect_list[new_key][anon_reviewer]:
                                    prev_date = inspect_list[new_key][anon_reviewer][asp][1]
                                    if prev_date < date_of_review:
                                        inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review]
                                else:
                                    inspect_list[new_key][anon_reviewer][asp] = [rev[asp], date_of_review]
            else:
                for asp in acl_aspects:
                    if asp in rev:
                        inspect_list.append((k, rev))

In [91]:
d = "16 Dec 2016 (modified: 20 Jan 2017)"
last_digits = re.findall('([0-9]+ [A-Za-z]+ [0-9][0-9][0-9][0-9]\))$', d)

In [98]:
inspect_list

{'abaselinefordetectingmisclassifiedandoutofdistributionexamplesinneuralnetworks': {'AnonReviewer1': {'MEANINGFUL_COMPARISON': [4,
    datetime.datetime(2016, 12, 19, 0, 0)],
   'ORIGINALITY': [4, datetime.datetime(2016, 12, 19, 0, 0)],
   'RECOMMENDATION': [6, datetime.datetime(2016, 12, 19, 0, 0)]},
  'AnonReviewer2': {'RECOMMENDATION': [6,
    datetime.datetime(2016, 12, 16, 0, 0)]},
  'AnonReviewer3': {'IMPACT': [2, datetime.datetime(2017, 1, 20, 0, 0)],
   'ORIGINALITY': [3, datetime.datetime(2017, 1, 20, 0, 0)],
   'RECOMMENDATION': [6, datetime.datetime(2017, 1, 20, 0, 0)],
   'SOUNDNESS_CORRECTNESS': [3, datetime.datetime(2017, 1, 20, 0, 0)]}},
 'acompareaggregatemodelformatchingtextsequences': {'AnonReviewer1': {'RECOMMENDATION': [7,
    datetime.datetime(2016, 12, 16, 0, 0)],
   'SOUNDNESS_CORRECTNESS': [5, datetime.datetime(2016, 12, 16, 0, 0)],
   'SUBSTANCE': [4, datetime.datetime(2016, 12, 16, 0, 0)]},
  'AnonReviewer2': {'ORIGINALITY': [3, datetime.datetime(2016, 12, 11,

In [30]:
aspect_dict

defaultdict(dict,
            {'abaselinefordetectingmisclassifiedandoutofdistributionexamplesinneuralnetworks': {'IMPACT': [2,
               2],
              'MEANINGFUL_COMPARISON': [4, 4, 4, 4],
              'ORIGINALITY': [4, 3, 4, 4, 3, 4],
              'RECOMMENDATION': [6, 6, 6, 6, 6, 6],
              'SOUNDNESS_CORRECTNESS': [3, 3],
              'dec': True,
              'pr_id': '410',
              'title': 'A Baseline for Detecting Misclassified and Out-of-Distribution Examples in Neural Networks'},
             'acompareaggregatemodelformatchingtextsequences': {'CLARITY': [5,
               5,
               5,
               5],
              'IMPACT': [3, 3, 3, 3],
              'ORIGINALITY': [3, 3, 3, 3],
              'RECOMMENDATION': [6, 7, 8, 6, 7, 8],
              'SOUNDNESS_CORRECTNESS': [5, 5],
              'SUBSTANCE': [4, 4],
              'dec': True,
              'pr_id': '363',
              'title': 'A Compare-Aggregate Model for Matching Text Seq

In [31]:
len(aspect_dict)

427

In [37]:
len(aspect_dict)

427

In [32]:
count = 0
for k in aspect_dict:
    if "APPROPRIATENESS" in aspect_dict[k]:
        count +=1
print(count)

13


In [33]:
len(inspect_list)

0

In [15]:
data_dir = "../ICLR data/masterdata_unbalanced/"
year = 2017
paper_file = "papers_{}.pkl".format(year)


with open(data_dir+paper_file, "rb") as f:
    papers_data = pickle.load(f)

In [16]:
len(papers_data)

490

In [17]:
papers_data["B1-Hhnslg"]["content"]["title"]

'Prototypical Networks for Few-shot Learning'

In [18]:
final_dict = {}

not_found = []

for k in papers_data:
    t = papers_data[k]["content"]["title"].strip().lower()
    t = re.sub('[^0-9a-zA-Z]+', '', t)
    if t in aspect_dict:
        final_dict[k] = aspect_dict[t]
    else:
        print(t)
        not_found.append(t)

shortanddeepsketchingandneuralnetworks
queryreductionnetworksforquestionanswering
learningtoplayinadayfasterdeepreinforcementlearningbyoptimalitytightening
progressiveattentionnetworksforvisualattributeprediction
representinginferentialuncertaintyindeepneuralnetworksthroughsampling
multiviewgenerativeadversarialnetworks
deeperrorcorrectingoutputcodes
neuralphotoeditingwithintrospectiveadversarialnetworks
normalizingthenormalizerscomparingandextendingnetworknormalizationschemes
bitpragmaticdeepneuralnetworkcomputing
rl2fastreinforcementlearningviaslowreinforcementlearning
generativemodelsandmodelcriticismviaoptimizedmaximummeandiscrepancy
hadamardproductforlowrankbilinearpooling
thepowerofsparsityinconvolutionalneuralnetworks
generativeadversarialnetworksasvariationaltrainingofenergybasedmodels
datanoisingassmoothinginneuralnetworklanguagemodels
deepsymbolicrepresentationlearningforheterogeneoustimeseriesclassification
adversarialexamplesinthephysicalworld

trackingtheworldstatewithrecu

In [19]:
len(not_found)

63

In [20]:
final_dict

{'B1-Hhnslg': {'RECOMMENDATION': [5, 6, 4, 5, 6, 4],
  'dec': False,
  'pr_id': '559',
  'title': 'Prototypical Networks for Few-shot Learning'},
 'B1-q5Pqxl': {'CLARITY': [5, 5, 5, 5, 5, 5, 5, 5],
  'IMPACT': [3, 3, 3, 3],
  'ORIGINALITY': [5, 5, 5, 5],
  'RECOMMENDATION': [6, 7, 6, 6, 7, 6],
  'SUBSTANCE': [5, 5, 5, 5],
  'dec': True,
  'pr_id': '384',
  'title': 'Machine Comprehension Using Match-LSTM and Answer Pointer'},
 'B16Jem9xe': {'RECOMMENDATION': [6, 8, 7, 6, 8, 7],
  'dec': False,
  'pr_id': '534',
  'title': 'Learning in Implicit Generative Models'},
 'B16dGcqlx': {'CLARITY': [5, 5, 5, 5],
  'ORIGINALITY': [1, 1, 1, 1],
  'RECOMMENDATION': [5, 6, 6, 5, 6, 6],
  'SOUNDNESS_CORRECTNESS': [4, 4, 4, 4],
  'dec': True,
  'pr_id': '341',
  'title': 'Third Person Imitation Learning'},
 'B184E5qee': {'CLARITY': [5, 5, 5, 5],
  'IMPACT': [2, 2, 2, 2],
  'ORIGINALITY': [2, 2, 2, 2],
  'RECOMMENDATION': [7, 5, 9, 7, 5, 9],
  'SOUNDNESS_CORRECTNESS': [4, 4, 4, 4],
  'dec': True,
  'p

In [21]:
with open("./features/rev_aspects_2017_peerread.pkl", "wb") as f:
    pickle.dump(final_dict, f)

In [22]:
ls features/

all_data_features_17_20.pkl          data_features_csv-2017.pkl
all_data_features_csv_17_20_new.pkl  features_csv_17_20.pkl
all_data_features_csv_2017_new.pkl   rev_aspects_2017_peerread.pkl
data_csv-2017-20.pkl


## Data Sanity Verification

In [23]:
with open("./features/rev_aspects_2017_peerread.pkl", "rb") as f:
    review_scores = pickle.load(f)

In [35]:
review_scores["B1-q5Pqxl"]

{'CLARITY': [5, 5, 5, 5, 5, 5, 5, 5],
 'IMPACT': [3, 3, 3, 3],
 'ORIGINALITY': [5, 5, 5, 5],
 'RECOMMENDATION': [6, 7, 6, 6, 7, 6],
 'SUBSTANCE': [5, 5, 5, 5],
 'dec': True,
 'pr_id': '384',
 'title': 'Machine Comprehension Using Match-LSTM and Answer Pointer'}

In [36]:
len(rr["384"]["reviews"])

18

In [40]:
pprint(rr["384"]["reviews"])

[{'IS_META_REVIEW': True,
  'comments': 'SUMMARY.\n'
              'This paper proposes a new neural network architectures for '
              'solving the task of reading comprehension question answering '
              'where the goal is answering a questions regarding a given text '
              'passage.\n'
              'The proposed model combines two well-know neural network '
              'architectures match-lstm and pointer nets.\n'
              'First the passage and the questions are encoded with a '
              'unidirectional LSTM.\n'
              'Then the encoded words in the passage and the encoded words in '
              'the questions are combined with an attention mechanism so that '
              'each word of the passage has a certain degree of compatibility '
              'with the question.\n'
              'For each word in the passage the word representation and the '
              'weighted representation of the query is concatenated and passed '
    

In [ ]:
'CLARITY': 5,
  'DATE': '17 Dec 2016',
  'IS_ANNOTATED': True,
  'IS_META_REVIEW': False,
  'OTHER_KEYS': 'ICLR 2017 conference AnonReviewer3',
  'RECOMMENDATION': 6,
  'REVIEWER_CONFIDENCE': 4,
  'SUBSTANCE': 5,
    
    
{'CLARITY': 5,
  'DATE': '03 Dec 2016',
  'IS_ANNOTATED': True,
  'IS_META_REVIEW': False,
  'OTHER_KEYS': 'ICLR 2017 conference AnonReviewer3',
  'SUBSTANCE': 5,
  'TITLE': 'Clarify contribution',
  'comments': ''},


{'CLARITY': 5,
  'DATE': '17 Dec 2016',
  'IS_ANNOTATED': True,
  'IS_META_REVIEW': False,
  'OTHER_KEYS': 'ICLR 2017 conference AnonReviewer3',
  'RECOMMENDATION': 6,
  'REVIEWER_CONFIDENCE': 4,
  'SUBSTANCE': 5,
  'TITLE': 'Review: Interesting combination of existing approaches with '
           'encouraging results',
  'comments': 'The paper looks at the problem of locating the answer to a '
              'question in a text (For this task the answer is always part of '
              'the input text). For this the paper proposes to combine two '}

{'CLARITY': 5,
  'DATE': '03 Dec 2016',
  'IS_ANNOTATED': True,
  'IS_META_REVIEW': False,
  'OTHER_KEYS': 'ICLR 2017 conference AnonReviewer3',
  'SUBSTANCE': 5,
  'TITLE': 'Clarify contribution',
  'comments': ''},
  

In [26]:
dt = parse('03 dec 2016')
dt1 = parse('03 Dec 2016')

In [12]:
print(dt)
print(dt.strftime('%d/%m/%Y'))

2016-12-03 00:00:00
03/12/2016


In [27]:
dt > dt1, dt < dt1, dt == dt1

(False, False, True)

In [28]:
dt, dt1

(datetime.datetime(2016, 12, 3, 0, 0), datetime.datetime(2016, 12, 3, 0, 0))